In [1]:
cd "/content/drive/My Drive/Colab Notebooks/Matrix/Matrix_three_road_signs_recognition/DW_Matrix_roadSignRecognition"

/content/drive/My Drive/Colab Notebooks/Matrix/Matrix_three_road_signs_recognition/DW_Matrix_roadSignRecognition


In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('bach_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
  )
  
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(num_classes, input_shape):
  return Sequential ([
     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
     MaxPool2D(),
     Dropout(0.3),

     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
     MaxPool2D(),
     Dropout(0.3),

     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
     MaxPool2D(),
     Dropout(0.3),

     Flatten(),

     Dense(1024, activation='relu'), 
     Dropout(0.3),

     Dense(1024, activation='relu'), 
     Dropout(0.3),

     Dense(num_classes, activation='softmax')
  ])



In [8]:
model = get_cnn_v5(num_classes, input_shape)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 24ms/step - loss: 2.4528 - accuracy: 0.3211 - val_loss: 1.1463 - val_accuracy: 0.6802
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 0.9573 - accuracy: 0.6977 - val_loss: 0.3491 - val_accuracy: 0.8986
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.5630 - accuracy: 0.8183 - val_loss: 0.1737 - val_accuracy: 0.9551
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3678 - accuracy: 0.8847 - val_loss: 0.1132 - val_accuracy: 0.9630
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2981 - accuracy: 0.9070 - val_loss: 0.0771 - val_accuracy: 0.9776


0.9551020408163265

In [11]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 4ms/step - loss: 0.1972 - accuracy: 0.9551


0.9551020264625549

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  

  model.fit(
      X_train, 
      y_train,
      batch_size = int( params.get('bach_size', 128) ),
      epochs = 5,
      verbose = 0      
  )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'modeml': model}

In [31]:
space = {
    'batch_size' : hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one' :  hp.uniform('dropout_cnn_block_one', 0, 1),
    'dropout_cnn_block_two' :  hp.uniform('dropout_cnn_block_two', 0, 1),
    'dropout_cnn_block_three' :  hp.uniform('dropout_cnn_block_three', 0, 1),

    'dropout_dense_block_one' :  hp.uniform('dropout_dense_block_one', 0, 1),
    'dropout_dense_block_two' :  hp.uniform('dropout_dense_block_two', 0, 1)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 140.0, 'dropout_cnn_block_one': 0.7192536492594018, 'dropout_cnn_block_three': 0.8527586215295189, 'dropout_cnn_block_two': 0.774314719245453, 'dropout_dense_block_one': 0.3326870200317217, 'dropout_dense_block_two': 0.21529260651476534}
accuracy=0.10544217377901077
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.765254128759447, 'dropout_cnn_block_three': 0.5406290471828296, 'dropout_cnn_block_two': 0.6080994958608368, 'dropout_dense_block_one': 0.34313575222015724, 'dropout_dense_block_two': 0.8597657922863989}
accuracy=0.1863945573568344
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.09130655452158087, 'dropout_cnn_block_three': 0.06209465259407265, 'dropout_cnn_block_two': 0.04695029472396994, 'dropout_dense_block_one': 0.8123698222846684, 'dropout_dense_block_two': 0.6253828974015545}
accuracy=0.9453514814376831
{'batch_size': 60.0, 'dropout_cnn_block_one': 0.41730493685133974, 'dropout_cnn_block_three': 0.9229685981863033, 'dropout_cnn_block_two': 0.179523909673

In [0]:
def get_model(params):
  return Sequential ([
     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
     Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
     MaxPool2D(),
     Dropout(params['dropout_cnn_block_one']),

     Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
     Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
     MaxPool2D(),
     Dropout(params['dropout_cnn_block_two']),

     Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
     Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
     MaxPool2D(),
     Dropout(params['dropout_cnn_block_three']),

     Flatten(),

     Dense(1024, activation='relu'), 
     Dropout(params['dropout_dense_block_one']),

     Dense(1024, activation='relu'), 
     Dropout(params['dropout_dense_block_two']),

     Dense(num_classes, activation='softmax')
  ])

